# Load and Process Graph Data

In this session, you will learn:

* Load graph data stored in CSV files.
* Construct a graph in DGL.
* Query structural information of a DGL graph.
* Load and pre-process node and edge features.


## Load graph data from CSV

Comma Separated Values (CSV) is a widely-used format for storing relational data. In this tutorial, we have prepared two CSV files that store [the Zachery's Karate Club network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).
* The `nodes.csv` stores every club members and their attributes.
* The `edges.csv` stores the pair-wise interactions between two club members.

In [ ]:
!ls -lh 'data'

We can use `pandas` to load the two CSV files.

In [ ]:
import pandas as pd

nodes_data = pd.read_csv('data/nodes.csv')
print(nodes_data)

In [ ]:
edges_data = pd.read_csv('data/edges.csv')
print(edges_data)

We then construct a graph where each node is a club member and each edge represents their interactions. In DGL, **nodes are consecutive integers starting from zero**. Thus, when preparing the data, it is important to re-label or re-shuffle the row order so that the first row corresponding to the first nodes, so on and so forth.

In this example, we have already prepared the data in the correct order, so we can create the graph by the `'Src'` and `'Dst'` columns from the `edges.csv` table.

In [ ]:
import dgl

src = edges_data['Src'].to_numpy()
dst = edges_data['Dst'].to_numpy()

# Create a DGL graph from a pair of numpy arrays
g = dgl.graph((src, dst))

# Print a graph gives some meta information such as number of nodes and edges.
print(g)

A DGL graph can be converted to a `networkx` graph, so to utilize its rich functionalities such as visualization.

In [ ]:
import networkx as nx
# Since the actual graph is undirected, we convert it for visualization
# purpose.
nx_g = g.to_networkx().to_undirected()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_g)
nx.draw(nx_g, pos, with_labels=True, node_color=[[.7, .7, .7]])

## Query graph structures

Let's print out how many nodes and edges are there in this graph.

In [ ]:
print('#Nodes', g.number_of_nodes())
print('#Edges', g.number_of_edges())

We can also perform queries on the graph structures.

Get the in-degree of node 0:

In [ ]:
g.in_degree(0)

Get the successors of node 0:

In [ ]:
g.successors(0)

DGL provides APIs for querying structural information. See the API document [here](https://docs.dgl.ai/api/python/heterograph.html#querying-graph-structure).

![image.png](../asset/dgl-query.png)

## Load node and edge features

In many graph data, nodes and edges have attributes. Although these attributes can have arbitrary types, a DGL graph only accepts attributes stored in tensors (with numerical contents). The vast development of deep learning has provided us many ways to vectorize various types of attributes into numerical features. Here are some general suggestions:
* For categorical attributes (e.g. gender, occupation), consider converting them to integers or one-hot encoding.
* For variable length string contents (e.g. news article, quote), consider applying a language model.
* For images, consider applying a vision model such as CNNs.

Our data set has the following attribute columns:
* `Age` is already an integer attribute.
* `Club` is a categorical attribute representing which community each member belongs to.
* `Weight` is a floating number indicating the strength of each interaction.

In [ ]:
import torch
import torch.nn.functional as F

# Prepare the age node feature
age = torch.tensor(nodes_data['Age'].to_numpy()).float() / 100
print(age)

In [ ]:
# Get the features of node 0 and 10
age[[0, 10]]

Use `g.ndata` to set the age features to the graph.

In [ ]:
# Feed the features to graph
g.ndata['age'] = age
print(g)

In [ ]:
# The "Club" column represents which community does each node belong to.
# The values are of string type, so we must convert it to either categorical
# integer values or one-hot encoding.

club = nodes_data['Club'].to_list()
# Convert to categorical integer values with 0 for 'Mr. Hi', 1 for 'Officer'.
club = torch.tensor([c == 'Officer' for c in club]).long()
# We can also convert it to one-hot encoding.
club_onehot = F.one_hot(club)
print(club_onehot)

# Use `g.ndata` like a normal dictionary
g.ndata.update({'club' : club, 'club_onehot' : club_onehot})
# Remove some features using del
del g.ndata['age']

print(g)

Feeding edge features to a DGL graph is similar.

In [ ]:
# Get edge features from the DataFrame and feed it to graph.
edge_weight = torch.tensor(edges_data['Weight'].to_numpy())
# Similarly, use `g.edata` for getting/setting edge features.
g.edata['weight'] = edge_weight
print(g)